In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from textblob import TextBlob
import json
import bert
import openai


A ideia deste projeto eh utilizar o bert para codificar a frase, utilizar PCA para reduzir sua dimensionalidade e depois usar knn para dizer se a palavra "servidor" trata-se de um hardware ou nao(polisemia). https://aclanthology.org/2022.deelio-1.2.pdf

Problemas: O dicionario do BERT nao contem palavras como "HP", "rack", etc. e por isso eh necessario fazer um fine tunning no tokenizer; a base de dados possui muitos labels como 0 e poucos como 1, e por isso nao eh uma base balanceada.

In [2]:
ex1 = "gerada pelo município de Tres Pontas/mg, que abrange Servidores públicos municipais ativos e inativos "
if ("servidor" in ex1 or "servidores" in ex1 or "Servidor" in ex1 or "Servidores" in ex1):
    for i, word in enumerate(ex1.split()):
        if word in ["servidor", "servidores", "Servidor","Servidores"]:
            print(i, word)

8 Servidores


In [5]:
from transformers import BertTokenizer, BertForPreTraining, AutoModel, AutoTokenizer

model_name  = 'neuralmind/bert-base-portuguese-cased'
#tokenizer = BertTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
new_tokens = ['Dell', 'HP','Lenovo', 'rack', 'Datacenter','NOBLEAK']
num_added_toks = tokenizer.add_tokens(new_tokens)
#model = BertForPreTraining.from_pretrained(model_name)
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

#tokenizer.tokenize('meus servidores não gostam de morangos.')
tokenizer.tokenize(ex1)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['gerada',
 'pelo',
 'município',
 'de',
 'Tre',
 '##s',
 'Ponta',
 '##s',
 '/',
 'mg',
 ',',
 'que',
 'abrange',
 'Servi',
 '##dores',
 'públicos',
 'municipais',
 'ativos',
 'e',
 'ina',
 '##tivos']

In [9]:
token2idx = tokenizer.get_vocab()
idx2token = {value:key for key, value in token2idx.items()}

print(token2idx['servidor'], token2idx['servidores'])
#print(token2idx['rack'], token2idx['HP'], token2idx['Lenovo'])
#print(idx2token[12879], idx2token[13291])

12879 13291


KeyError: 'rack'

In [24]:
#Este metodo nao funciona bem. Melhor arrumar as labels manualmente.
#Este bloco e o proximo sao para fazer uma requisicao GET no chatgpt.
with open('Servidor/data1.json', 'r') as fp:
    file_contents = json.load(fp)
data = []
for x in file_contents:
    if (("servidor" in x['Descrição Objeto'] or "servidores" in x['Descrição Objeto'] or "Servidor" in x['Descrição Objeto'] or "Servidores" in x['Descrição Objeto']) and ("data center" in x['Descrição Objeto'] or "Datacenter" in x['Descrição Objeto'] or "NOBLEAK" in x['Descrição Objeto'] or "HP" in x['Descrição Objeto'] or "Dell" in x['Descrição Objeto'] or "rack" in x['Descrição Objeto'])):
        data.append({"text": x['Descrição Objeto'], "label":1})
        print(f"O servidor da licitacao de codigo {x['Código ']} trata-se de um hardware")
    else:
        data.append({"text": x['Descrição Objeto'], "label":0})
print(data[0:2])

[{'text': 'Despesas com a contratação da Companhia para a implementação de software para gerenciamento de documentos eletrônicos com departamentos, treinamento de servidores e suporte técnico, com a câmera municipal de IPOA, Estado de Goias.', 'label': 0}, {'text': 'Equipamento de processamento de dados, G15-I1200-M40P 15.6 "FHD 12A Intel Core i7 de 16 GB de geração', 'label': 0}]


In [1]:
openai.my_api_key = 'YOUR_API_KEY'
response = {}
datagpt = [x['Descrição Objeto'] for x in file_contents]
for i, answer in enumerate datagpt:
    if ("servidor" in answer or "servidores" in answer or "Servidor" in answer or "Servidores" in answer):  
        prompt = answer + 'Na frase anterior, "servidor" refere-se a uma pessoa ou a um hardware?'
        completition = openai.ChatCompletition(model="gpt-3.5-turbo", message = prompt)
        reply = completition.choices[0].message.content
        response[str(i)] = reply

Note: you may need to restart the kernel to use updated packages.


In [26]:
#data_inputs = tokenizer(data, return_tensors='pt', max_length=512, truncation=True, padding='max_length') 
data_inputs = [{'text':tokenizer(sent["text"], return_tensors='pt', max_length=512, truncation=True, padding='max_length'), 'label': sent['label']} if sent['label'] == 0 else {'text': sent['text'], 'label': sent['label']}for sent in data ]
print(len(data_inputs))
print(data_inputs[0]['text']['input_ids'].shape)

30
torch.Size([1, 512])


In [35]:
print(data_inputs)

[{'text': {'input_ids': tensor([[  101,  1305, 14217,   170,   123, 12782,   180,  4796,   221,   123,
         10574,   125,  6545,   221, 15573,   125,  5590, 10748,   170, 14494,
           117,  6992,   125, 13291,   122,  6405,  4022,   117,   170,   123,
         10299,  4858,   125, 11285, 22317, 22301,   117,  1477,   125,  1828,
           562,   119,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0

In [39]:
import torch
#checar se a palavra "servidor" aparece e nao se trata de um hardware
serv = [torch.tensor(12879),torch.tensor(13291)]#tokens para servidor e servidores
print(serv)
for data in data_inputs:
    serv_pos = []
    if(data['label'] == 0):
        for i,x in enumerate(data['text']['input_ids'][0]):
            if x in serv:
                serv_pos.append(i)
        print('------------------')
        if len(serv_pos) > 0:
            with torch.no_grad():
                outs = model(data['text']['input_ids'])
                encoded = outs[0][0]
                print(encoded)
                print(encoded.shape)
                for j in range(len(serv_pos)):
                    print(encoded[serv_pos[j]].shape)

[tensor(12879), tensor(13291)]
------------------
tensor([[ 0.1026,  0.6470,  0.5134,  ..., -0.0793, -0.0200, -0.9745],
        [-0.0492,  0.6545,  0.7757,  ...,  0.2884, -0.1914, -0.7993],
        [ 0.1106,  0.6323,  0.6213,  ...,  0.2067, -0.2317, -0.6857],
        ...,
        [-0.0207,  0.9627,  0.6509,  ...,  0.7290, -0.2480, -1.0775],
        [ 0.0022,  0.9051,  0.6838,  ...,  0.5886, -0.2936, -1.1149],
        [ 0.0659,  0.6825,  0.4971,  ...,  0.4948, -0.2480, -0.9504]])
torch.Size([512, 768])
torch.Size([768])
------------------
------------------
tensor([[-1.6000e-02,  5.0238e-01,  6.3186e-01,  ..., -4.8330e-02,
          1.5014e-01, -8.4201e-01],
        [ 1.8439e-01,  3.7730e-02, -1.8586e-02,  ...,  1.1150e-01,
         -9.4418e-02, -2.5426e-01],
        [ 1.9547e-01,  6.6389e-02,  4.6448e-02,  ..., -7.0053e-01,
          6.6913e-02, -1.4483e-01],
        ...,
        [-2.0059e-01,  8.6047e-01,  9.2123e-01,  ...,  6.2214e-01,
          9.4809e-02, -9.4625e-01],
        [-1.

tensor([[-2.9956e-02,  4.8401e-01,  6.8600e-01,  ...,  7.6685e-03,
          1.9071e-01, -8.3662e-01],
        [-2.5431e-01,  3.4636e-01,  1.7534e-01,  ...,  2.6668e-01,
         -1.4502e-01, -5.9857e-01],
        [-3.5273e-01,  3.2533e-01,  4.0350e-01,  ..., -3.5422e-01,
          7.1405e-02, -8.7020e-01],
        ...,
        [-2.2973e-01,  8.5495e-01,  8.7031e-01,  ...,  6.5291e-01,
          7.6505e-02, -9.6238e-01],
        [-2.0831e-01,  7.9980e-01,  8.7077e-01,  ...,  4.7719e-01,
          3.6316e-02, -1.0225e+00],
        [-3.2409e-04,  5.7183e-01,  7.0015e-01,  ...,  3.4712e-01,
         -5.4877e-02, -8.6527e-01]])
torch.Size([512, 768])
torch.Size([768])
------------------
